In [2]:
%pip install dgl

     |████████████████████████████████| 6.3 MB 399 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import dgl
import torch as th
# edges 0->1, 0->2, 0->3, 1->3
u, v = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
g = dgl.graph((u, v))
print(g) # number of nodes are inferred from the max node IDs in the given edges
# Node IDs
print(g.nodes())
# Edge end nodes
print(g.edges())
# Edge end nodes and edge IDs
print(g.edges(form='all'))
# If the node with the largest ID is isolated (meaning no edges),
# then one needs to explicitly set the number of nodes
g = dgl.graph((u, v), num_nodes=8)

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={})
tensor([0, 1, 2, 3])
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]))
(tensor([0, 0, 0, 1]), tensor([1, 2, 3, 3]), tensor([0, 1, 2, 3]))


In [3]:
edges = th.tensor([2, 5, 3]), th.tensor([3, 5, 0])  # 边：2->3, 5->5, 3->0
g64 = dgl.graph(edges)  # DGL默认使用int64
print(g64.idtype)
g32 = dgl.graph(edges, idtype=th.int32)  # 使用int32构建图
g32.idtype
# g64_2 = g32.long()  # 转换成int64
# g64_2.idtype
# g32_2 = g64.int()  # 转换成int32
# g32_2.idtype

torch.int64


torch.int32

In [5]:
import dgl
import torch as th
g = dgl.graph(([0, 0, 1, 5], [1, 2, 2, 0])) # 6个节点，4条边
g
g.ndata['x'] = th.ones(g.num_nodes(), 3)               # 长度为3的节点特征
g.edata['x'] = th.ones(g.num_edges(), dtype=th.int32)  # 标量整型特征
g
# 不同名称的特征可以具有不同形状
g.ndata['y'] = th.randn(g.num_nodes(), 5)
# print(g.ndata['y'])
g.ndata['x'][1]                  # 获取节点1的特征
g.edata['x'][th.tensor([0, 3])]  # 获取边0和3的特征

tensor([[ 1.4918,  0.2774, -0.1152, -1.0215, -1.4262],
        [-2.6538,  0.3229,  0.9344, -1.1643, -1.0056],
        [ 0.3081, -0.6746,  1.5852,  0.4632,  1.8549],
        [ 0.4038, -1.0142, -0.2733, -0.2340,  0.8484],
        [ 1.4929, -0.7657,  0.1483, -1.1800, -1.5759],
        [-1.3395, -0.6130, -0.6207,  0.8257,  0.9696]])


tensor([1, 1], dtype=torch.int32)

In [5]:
# 边 0->1, 0->2, 0->3, 1->3
edges = th.tensor([0, 0, 0, 1]), th.tensor([1, 2, 3, 3])
weights = th.tensor([0.1, 0.6, 0.9, 0.7])  # 每条边的权重
g = dgl.graph(edges)
g.edata['w'] = weights  # 将其命名为 'w'
g

Graph(num_nodes=4, num_edges=4,
      ndata_schemes={}
      edata_schemes={'w': Scheme(shape=(), dtype=torch.float32)})

In [11]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
4


In [18]:
# import dgl
# import torch as th
# u, v = th.tensor([0, 1, 2]), th.tensor([2, 3, 4])
# g = dgl.graph((u, v))
# g.ndata['x'] = th.randn(5, 3)   # 原始特征在CPU上
# g.device
# cuda_g = g.to('cuda:1')         # 接受来自后端框架的任何设备对象
# cuda_g.device
# cuda_g.ndata['x'].device        # 特征数据也拷贝到了GPU上
# # 由GPU张量构造的图也在GPU上
# u, v = u.to('cuda:0'), v.to('cuda:0')
# g = dgl.graph((u, v))
# g.device


import dgl
import torch as th

device = 'cuda:1' if th.cuda.is_available() else 'cpu'

u, v = th.tensor([0, 1, 2]).to(device), th.tensor([2, 3, 4]).to(device)
g = dgl.graph((u, v))
g.ndata['x'] = th.randn(5, 3).to(device)

print(g.device)  # This should now print the GPU device
print(g.ndata['x'].device)  # This should also print the GPU device


cuda:1
cuda:1


In [15]:
g.in_degrees()
g.in_edges([2, 3, 4])                          # 可以接受非张量类型的参数
g.in_edges(th.tensor([2, 3, 4]).to('cuda:0'))  # 张量类型的参数必须在GPU上
g.ndata['h'] = th.randn(5, 4)                  # ERROR! 特征也必须在GPU上！

DGLError: [16:08:10] /opt/dgl/src/array/array.cc:764: Operator COOToCSR does not support cuda device.
Stack trace:
  [bt] (0) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(+0xf87d0) [0x7f5c276817d0]
  [bt] (1) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(dgl::aten::COOToCSR(dgl::aten::COOMatrix)+0x14d) [0x7f5c276933dd]
  [bt] (2) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(dgl::UnitGraph::GetInCSR(bool) const+0x3ac) [0x7f5c27ba504c]
  [bt] (3) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(dgl::UnitGraph::GetFormat(dgl::SparseFormat) const+0x4d) [0x7f5c27ba5fbd]
  [bt] (4) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(dgl::UnitGraph::InDegrees(unsigned long, dgl::runtime::NDArray) const+0x36) [0x7f5c27ba7386]
  [bt] (5) /workdir/home/euni/anaconda3/lib/python3.9/site-packages/dgl/libdgl.so(dgl::HeteroGraph::InDegrees(unsigned long, dgl::runtime::NDArray) const+